# Simulating a Quantum Computer in Python

In [2]:
import numpy as np
import numpy.linalg as lg
import numpy.random as random

In [3]:
N_qubits = 3
N_states = 2**N_qubits

Create a quantum register

In [4]:
def empty_register():
    return np.zeros((N_states), dtype=complex)

In [5]:
register = empty_register()
register

array([ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
        0.+0.j])

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}\newcommand{\bra}[1]{\left\langle{#1}\right|}$
The states of our 3-qubit quantum register are $\ket{000}$, $\ket{001}$, $\ket{010}$, ..., which we will address as state 0, 1,  2, ...

We define two helper functions. The first one checks if a certain qubit is set to $1$ in a state. The second one flips a target qubit from $0$ to $1$ or from $1$ to $0$. In the $\ket{011}$ notation, the 0-th qubit is the leftmost one, the $N_{qubit}-1$-th qubit is the rightmost one.

In [6]:
def isset(state, n):
	return state & 1<<(N_qubits-1-n) != 0

def flip(state, n):
	return state ^ 1<<(N_qubits-1-n)

In [7]:
print bin(1)
isset(1,2)

0b1


True

In [8]:
isset(flip(8,2),2)

True

In [9]:
def measure(register):
	register = np.absolute(register)**2
	return random.choice(len(register), p=register.flatten())

In [22]:
def hadamar(register):
	H_2x2 = np.array([[1,1],[1,-1]]) / np.sqrt(2)
	H = np.array([[1,1],[1,-1]]) / np.sqrt(2)
	for i in range(N_qubits-1):
		H = np.kron(H,H_2x2)
	print H
	return H.dot(register)

In [11]:
def cnot(register, control, target):
	CNOT = np.zeros((N_states, N_states))
	for state in range(N_states):
		if isset(state, control): # control state set to 1
			target_state = flip(state, target) # flip target bit
		else:
			target_state = state
		CNOT[target_state, state] = 1
	return CNOT.dot(register)

In [12]:
def pauli_x(register, target):
	for state in range(N_states):
		if isset(state, target):
			conjugated_state = flip(state, target)
			tmp = register[state]
			register[state] = register[conjugated_state]
			register[conjugated_state] = tmp
	return register

In [13]:
def pauli_y(register, target):
	for state in range(N_states):
		if isset(state, target):
			conjugated_state = flip(state, target)
			tmp = register[state]
			register[state] = -1j * register[conjugated_state]
			register[conjugated_state] = 1j * tmp
	return register

In [14]:
def pauli_z(register, target):
	for state in range(N_states):
		if isset(state, target):
			register[state] = -1 * register[state]
	return register

In [15]:
def phase_shift(register, target, phase):
	for state in range(N_states):
		if isset(state, target):
			register[state] = np.exp(-1j*phase) * register[state]
	return register

In [16]:
def phase(register, target):
	for state in range(N_states):
		if isset(state, target):
			register[state] = 1j * register[state]
	return register

## Groover Algorithm

In [17]:
def conditional_phase_shift(register):
	for state in range(1,N_states):
		register[state] = -1 * register[state]
	return register

The oracle function is a unitary transformation which shifts the specific of the seeked state by 2π. Applying the oracle to a register with equally distributed real and positive amplitudes will therefore switch the sign of one of the states. Note that the specific probabilities of the states do not change since they are related to the squared amplitues.

In [18]:
def oracle(register):
	correct_state = 5
	register[correct_state] = -1 * register[correct_state]
	return register

In [23]:
print "Initializing ground state"

# Initialize ground state
register = np.zeros((N_states), dtype=complex)
register[0] = 1.0
print register
print "Hadamar transformation"
register = hadamar(register)
print register

cycles = np.pi/4*np.sqrt(2**N_qubits)
print "Grover iteration cycles", cycles

for i in range(int(np.round(cycles))):
	print "Iteration", i

	print "Oracle application"
	register = oracle(register)
	#print register

	print "Diffusion transform"
	register = hadamar(register)
	register = conditional_phase_shift(register)
	register = hadamar(register)
	print register

print "Measurement"
print measure(register)

Initializing ground state
[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
Hadamar transformation
[[ 0.35355339  0.35355339  0.35355339  0.35355339  0.35355339  0.35355339
   0.35355339  0.35355339]
 [ 0.35355339 -0.35355339  0.35355339 -0.35355339  0.35355339 -0.35355339
   0.35355339 -0.35355339]
 [ 0.35355339  0.35355339 -0.35355339 -0.35355339  0.35355339  0.35355339
  -0.35355339 -0.35355339]
 [ 0.35355339 -0.35355339 -0.35355339  0.35355339  0.35355339 -0.35355339
  -0.35355339  0.35355339]
 [ 0.35355339  0.35355339  0.35355339  0.35355339 -0.35355339 -0.35355339
  -0.35355339 -0.35355339]
 [ 0.35355339 -0.35355339  0.35355339 -0.35355339 -0.35355339  0.35355339
  -0.35355339  0.35355339]
 [ 0.35355339  0.35355339 -0.35355339 -0.35355339 -0.35355339 -0.35355339
   0.35355339  0.35355339]
 [ 0.35355339 -0.35355339 -0.35355339  0.35355339 -0.35355339  0.35355339
   0.35355339 -0.35355339]]
[ 0.35355339+0.j  0.35355339+0.j  0.35355339+0.j  0.35355339+0.j
  0.3535533

## Simon's Algorithm

Given a function
$$f: \{0,1\}^n \rightarrow \{0,1\}^n $$
that is known to be invariant under some bitmask $a$, determine $a$.

We start by initializing two quantum registers in the state $\ket{0}$

In [37]:
N_qubits = 6
N_states = 2**N_qubits

In [38]:
register = empty_register()

In [47]:
def str2state(string):
    return int(string,2)
def state2str(state):
    return str(bin(state))[2:]

The hadamar transform is applied to the first register to get an equal superposition of states in the quantum register.

In [49]:
register[:2**3] = 1/np.sqrt(8)
register

array([ 0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,
        0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  

Next, the oracle is applied on both registers. It is implemented as a unitary transform, which performs the following calculation:

$$\mathcal O _{f(x)} = \ket{x} \ket{y} = \ket{x} \ket{f(x) \oplus y} $$

In [ ]:
def simon_oracle(register0, register1):
    table = {0:5, 1:2, 2:0, 3:5, 4:0, 5:6, 6:5, 7:2}
    mask = 6
    for state in range(N_states):
        register1[state] = table[register0] ^ mask
    return register0, register 1